↑プロジェクトトークンを挿入したら、実行を忘れずに!

# Db2でシェアサイクルポートの地理情報分析をやってみよう!
## データセットアップ


**実行は　セルを選択して**

- **Windows： Ctrl+Enter**
- **Mac: ⌘ (command) + Enter 　または Ctrl + Enter**


### 前提
 - [地理空間分析機能を有効にしている](https://qiita.com/nishikyon/items/4cd0430e4ebb7d541a82)
 - [JGD2011座標系の設定済み](https://qiita.com/nishikyon/items/fc97391f02b4e9e4bb9c)
 
   Watson Studio上のnotebookからIBM Cloud Object Storage(ICOS)へのFileの読み書き - project-libを使う -の手順が前準備として必要です。<br>
   <br>
    まずは1,2を実施お願いします。<br>
    
    - 1.[ Projectを開く](https://qiita.com/nishikyon/items/1bed62a2a98b0c970f40#1-project%E3%82%92%E9%96%8B%E3%81%8F)
    - 2.[ 前準備アクセス・トークンの作成](https://qiita.com/nishikyon/items/1bed62a2a98b0c970f40#2-%E5%89%8D%E6%BA%96%E5%82%99%E3%82%A2%E3%82%AF%E3%82%BB%E3%82%B9%E3%83%88%E3%83%BC%E3%82%AF%E3%83%B3%E3%81%AE%E4%BD%9C%E6%88%90)
    <br>
    その後、このnotebookに
    <br>
    - [5: プロジェクト・トークンの挿入](https://qiita.com/nishikyon/items/1bed62a2a98b0c970f40#5-%E3%83%97%E3%83%AD%E3%82%B8%E3%82%A7%E3%82%AF%E3%83%88%E3%83%88%E3%83%BC%E3%82%AF%E3%83%B3%E3%81%AE%E6%8C%BF%E5%85%A5)を実施してください。
    <br>
    一番上に挿入されたセルの実行は忘れずにお願いします。
 
 2022/09/28のDataBase Dojoの環境で使用している場合は上記は既に設定済みです。


### 確認済み動作環境
- Watson Studio SaaS版

### ローカルで動作させる場合：
- OSコマンドが基本bashベースなので、Mac＆Linux向けです。Windowsで動作させるにはその部分の変更が必要です。
- ObjrectStorage上のファイル出力部分は要変更になります。





## 1. 前準備
### 1.1 Db2接続情報のセット

XXXXと9999は自分の接続先のDb2の情報を入れてください

In [ ]:
user = "xxxx" # db2ユーザーid
password = "xxxx" #db2パスワードを入れる
databese = "xxxx" # db2 Db名
host =  "xxxx" # db2 ホスト名
port =  9999 #Db2ポート番号


### 1.2 Db2 Jupyter Notebook Extensions のロード
https://github.com/IBM/db2-jupyter

notebookでdb2 SQLを使いやすくる`Db2 Jupyter Notebook Extensions`をダウンロード&ロードします。<br>
(使い方の参考: https://speakerdeck.com/kyokonishito/db2-and-jupyter-notebooks)

In [ ]:
import os
path = "./db2.ipynb"
if not os.path.exists(path):
    !wget https://raw.githubusercontent.com/IBM/db2-jupyter/master/db2.ipynb
else:
    print("db2.ipynb is existing.")
srs_id = None

In [ ]:
%run db2.ipynb

### 1.3 Db2に接続します
`Db2 Jupyter Notebook Extensions`を使用してDb2に接続

In [ ]:
%sql CONNECT TO {databese} USER {user} USING {password} HOST {host} port {port} SSL TRUE

## 2. 国土交通省国土数値情報ダウンロードサービスの東京都の行政区域データのSHAPEファイルを入手/加工
国土交通省が行政区域などのSHAPEファイルを作成して公開しています。

これを利用すると、例えば中央区の境界SHAPEデータ(Polygonデータ）などを入手することができます。

ここでは[国土交通省国土数値情報ダウンロードサービス](https://nlftp.mlit.go.jp/ksj/index.html)の東京都の行政区域データをダウンロードします。

正式に以下のようにはWeb GUIでダウンロードします。
「2.政策地域」の中の「行政区域（ポリゴン） 」
の「東京都」、「令和3年」のデータをダウンロードします。

(令和4年はうまくできなかったので、令和3年をダウンロードします)

ですが、notebookに直で使えるように、ここではwgetでダウンロード先から直接ダウンロードします。

また、このデータの文字コードはSJISです。Db2の文字コードがUTF-8の場合(Db2 Warehouse on Cloudの文字コードはUTF-8)、あとでツールを使ってDb2にSHAPEファイルをインポートする際に文字コードの指定ができないため、UTF-8に変換しないと文字化けします。

ここではgeopandasを使用してUTF-8に変換します。

- - -
注：2022/09/28のDataBase Dojoの環境で使用している場合<br>
ここで作成したファイルは「3. SHAPEファイルのインポート」で使用するのですが、SHAPEファイルのインポートはnotebookではできないためインポートは実施ぜず、あらかじめ作成済みのテーブルを使用します。よって2022/09/28のDataBase Dojoの環境で使用している場合はここをSKIPしても問題ありません。

### 2.1 東京都の行政区域データのSHAPEファイルのダウンロード

In [ ]:
#[国土交通省国土数値情報ダウンロードサービス](https://nlftp.mlit.go.jp/ksj/index.html)
#東京都の行政区域データ
#東京 世界測地系 令和3年 14.27MB N03-20210101_13_GML.zip
!wget https://nlftp.mlit.go.jp/ksj/gml/data/N03/N03-2021/N03-20210101_13_GML.zip

### 2.2 ダウンロードしたファイルの確認
`N03-20210101_13_GML.zip`があることを確認

In [ ]:
!ls *.zip

### 2.3 ダウンロードしたファイルを解凍

In [ ]:
!unzip -o N03-20210101_13_GML.zip

### 2.4 必要なパッケージを導入

[geopandas](https://geopandas.org/en/stable/)ライブラリは東京都の行政区域データのSJISデータをUTF-8に変換するのに使用します。

In [ ]:
!pip install geopandas

### 2.5 geopandasのGeoDataFrameにSHAPEファイルを読み込む

In [ ]:
import pandas as pd
import geopandas as gpd

file_path = r"./N03-20210101_13_GML/N03-21_13_210101.shp" #ダウンロードして解凍したSHAPEファイル
geo_df =gpd.read_file(file_path, encoding='shift-jis') # encodingはshift-jis
geo_df.head() #最初の5行の表示

### 2.6 読み込んだデータの座標参照系（CRS:Coordinate Reference System）を確認します。

In [ ]:
geo_df.crs

↑EPSG:6668のJGD2011であることがわかります。

[座標参照系とは?(ググります)](https://www.google.com/search?q=%E5%BA%A7%E6%A8%99%E5%8F%82%E7%85%A7%E7%B3%BB)

Db2 Spatial Anlaysisには事前によく使いそうな空間参照系の定義が<br>
ビュー`SYSGEO.ST_SPATIAL_REFERENCE_SYSTEMS`<br>
にあり、ここにあるもののみが、空間参照系として指定できます。

ここにない場合は、<br>
ビュー`SYSGEO.ST_COORDINATE_SYSTEMS`<br>
に5000 を超える座標系が用意されており、そのいずれかに基づいて新しい空間参照系を作成可能です。

残念ながらJGD2011の空間参照系は初期状態では`SYSGEO.ST_SPATIAL_REFERENCE_SYSTEMS`に定義されていません。<br>
よって、前準備として、「[JGD2011の空間参照系を設定](https://qiita.com/nishikyon/items/fc97391f02b4e9e4bb9c)」としています。<br>
またはSHAPEファイルの座標系を現在SYSGEO.ST_SPATIAL_REFERENCE_SYSTEMSに定義されている座標系に変換してもOKです(ここではやりません)。

以下でJGD2011の空間参照系の設定を確認します。
SRS_IDを変数srs_idに入れています。

In [ ]:
crs_df = %sql SELECT * FROM SYSGEO.ST_SPATIAL_REFERENCE_SYSTEMS WHERE ORGANIZATION='EPSG' AND ORGANIZATION_COORDSYS_ID = 6668;
#srs_idを取得
srs_id = crs_df['SRS_ID'][0]
print("srs_id is " +str(srs_id))
crs_df

### 2.6 読み込んだデータをUTF-8の文字コードで保存

`tokyo.shp` という名前で保存します。

In [ ]:
!rm -f tokyo.*
out="./tokyo.shp"
geo_df.to_file(out, encoding='utf-8')

### 2.7 確認
以下の5つのファイルが作成されていることを確認します：<br>

`tokyo.cpg`  `tokyo.dbf` `tokyo.prj` `tokyo.shp` `tokyo.shx`

In [ ]:
!ls tokyo*

### 2.8 まとめて1つのzipファイル作成

以下の5つのファイルをまとめて1つのzipファイル作成します：<br>

`tokyo.cpg`  `tokyo.dbf` `tokyo.prj` `tokyo.shp` `tokyo.shx`

In [ ]:
!zip -r tokyo.zip tokyo.*

### 2.9 プロジェクト資産に保存
ダウンロードできるようにプロジェクト資産に保存します。

In [ ]:
with open('tokyo.zip','rb') as f:
    project.save_data('tokyo.zip', f, overwrite=True)

## 3. SHAPEファイルのインポート
SHAPEファイルのインポートはこのnotebookではできません。
プロジェクト資産から`tokyo.zip`をダウンロードし、<br>
Qiita　[Db2地理空間分析: SHAPEファイルのインポート](https://qiita.com/nishikyon/items/fd6d166fdbf6523930d4)を参照してインポートしてください。

このnotebookと次に使用するnotebookではCITY_SHAPEテーブルにインポートしたとして進めます。

2022/09/28のDataBase Dojoの環境で使用している場合は上記はCITY_SHAPEビューを作成済みです。
このまま進めてください。

### 3.1 インポート後の確認
SHAPEデータがロードされた`CITY_SHAPE`の中身を見てみます

In [ ]:
# テーブルの構成確認
%sql DESCRIBE TABLE CITY_SHAPE

In [ ]:
#中身をちょっと確認
%sql SELECT N03_001, N03_002, N03_004, N03_007, OBJECTID, CAST( ST_AsText(GEO) AS VARCHAR(50) ) AS GEO from CITY_SHAPE FETCH FIRST 10 ROWS ONLY

# 4. IBM事業所情報のインポート
次の「地理空間分析」で使うので、こちらもテーブルを作成して、情報を入れておきます。

テーブル`IBM_LOC`に経度・緯度情報とともにIBM事業所情報が入ります。

ST_Point(<緯度>, <経度>, <空間参照系 ID >)で`ST_Point`データタイプに変換して、INSERTしています。

今回は、最初の方に確認しているJGD2011の空間参照系を使うので、変数に入れたsrs_idを使います。

In [ ]:
%%sql
DROP TABLE "IBM_LOC";
CREATE TABLE "IBM_LOC"  (
    "CODE" CHAR(30 OCTETS) NOT NULL, 
    "BRANCH_NAME" VARCHAR(60 OCTETS) NOT NULL, 
    "ADDRESS" VARCHAR(120 OCTETS) , 
    "GEO_POINT" ST_Point,
    PRIMARY KEY (CODE))  
IN "USERSPACE1" 

In [ ]:
import pandas as pd

ibm_loc_list = [["HQ","本社","東京都中央区日本橋箱崎町19-21",139.7869482,35.67833],
["GT","五反田","東京都品川区西五反田1-1-8 NMF五反田駅前ビル",139.7209161,35.6258705],
[ "MIT","三鷹駅前","東京都武蔵野市中町一丁目9番5号 第一中央ビル3階",139.5607469,35.704128],
["FC","府中","東京都府中市日鋼町1-1 ヒューリック府中タワー10F",139.47222,35.674475],
["TAMA","多摩","東京都多摩市落合1-15-2 多摩センタートーセイビル",139.428773,35.625697]]

col_name = ["CODE","BRUNCH_NAME","ADDRESS","LON","LAT"]

if srs_id is None:
    srs_id = 2011

ibm_loc_df = pd.DataFrame(data=ibm_loc_list,  columns=col_name)
sql_flag = '-e'
for i, row in ibm_loc_df.iterrows():
    code=row['CODE']
    branch_name=row['BRUNCH_NAME']
    address=row['ADDRESS']
    lon=row['LON']
    lat=row['LAT']
    print(branch_name)
    %sql {sql_flag} INSERT INTO IBM_LOC(CODE, BRANCH_NAME, ADDRESS, GEO_POINT) VALUES (:code, :branch_name, :address, ST_Point(:lon, :lat, {srs_id}))
    sql_flag = '-q'
    if sqlcode != 0:
        print("Line {0} Error : sqlcode:{1}, sqlstate:{2}, Message: {3}".format(i, sqlcode, sqlstate, sqlerror))
        break;

### 中身を確認します

In [ ]:
%sql select * from IBM_LOC

# 5. シェアサイクルポート情報のインポート

[公共交通オープンデータセンター](https://www.odpt.org/)の以下の2つのデータを1つのテーブル`SHAREBIKE_STATION`にインポートします。

- [ドコモ・バイクシェア バイクシェア関連情報(GBFS形式)](https://ckan.odpt.org/dataset/c_bikeshare_gbfs-d-bikeshare)の[バイクシェア関連情報 station_information](https://ckan.odpt.org/dataset/c_bikeshare_gbfs-d-bikeshare/resource/e8936a97-c341-416c-a39a-258335580de5)
- [OpenStreet（ハローサイクリング） バイクシェア関連情報(GBFS形式)](https://ckan.odpt.org/dataset/c_bikeshare_gbfs-openstreet)の[バイクシェア関連情報(GBFS形式 / station_information)](https://ckan.odpt.org/dataset/c_bikeshare_gbfs-openstreet/resource/5455c77d-154d-4b22-b0ca-9ea78209ed76)


参考: Qiita[公共交通オープンデータセンターにあるドコモ・バイクシェアのstation_information情報をCSVファイルにする](https://qiita.com/nishikyon/items/bfb328bca59c64228474)


### 5.1 SHAREBIKE_STATIONテーブルの作成

以下のDDLで作成します。`GEO_POINT`に経度・緯度情報を`ST_Point`データタイプで入れます。

In [ ]:
%%sql
DROP TABLE "SHAREBIKE_STATION";
CREATE TABLE "SHAREBIKE_STATION"  (
    "STATION_ID" VARCHAR(10 OCTETS) NOT NULL, 
    "REGION_ID" VARCHAR(10 OCTETS) , 
    "CAPACITY" INT , 
    "NAME" VARCHAR(120 OCTETS) ,
    "ADDRESS"   VARCHAR(300 OCTETS) ,
    "URL"  VARCHAR(300 OCTETS) ,
    "COMPANY"   VARCHAR(60 OCTETS) ,    
    "GEO_POINT" ST_Point,
    PRIMARY KEY (STATION_ID)) ;

### 5.2 ドコモ・バイクシェア バイクシェア関連情報  station_informationの取得
[公共交通オープンデータセンター](https://www.odpt.org/)から<br>
- [ドコモ・バイクシェア バイクシェア関連情報(GBFS形式)](https://ckan.odpt.org/dataset/c_bikeshare_gbfs-d-bikeshare)の[バイクシェア関連情報 station_information](https://ckan.odpt.org/dataset/c_bikeshare_gbfs-d-bikeshare/resource/e8936a97-c341-416c-a39a-258335580de5)
をダウンロードし、pandasのDataFrameに入れます。

In [ ]:
## ドコモ・バイクシェア バイクシェア関連情報 station_information　緯度・経度情報を取得
import requests
import json
import pandas as pd

#ドコモ・バイクシェア バイクシェア関連情報  station_information
req_url = 'https://api-public.odpt.org/api/v4/gbfs/docomo-cycle-tokyo/station_information.json'

# URLにアクセス
request = requests.get(req_url)

# 必要な部分のみ抜き出し
data_json = json.loads(request.text)['data']['stations']

# pandasのDataFrameに入れる
domomo_df = pd.DataFrame(data=data_json)
domomo_df

ドコモ・バイクシェア バイクシェア関連情報 station_informationには住所情報がありません。

### 5.3ドコモ・バイクシェア バイクシェア関連情報  station_informationのSHAREBIKE_STATIONへのインポート


ST_Point(<緯度>, <経度>, <空間参照系 ID >)で`ST_Point`データタイプに変換して、INSERTしています。

今回は、最初の方に確認しているJGD2011の空間参照系を使うので、変数に入れたsrs_idを使います。


In [ ]:
# SHAREBIKE_STATIONへのインポート(といいつつINSERT)
# 30秒から50秒かかります(WatssonStudio SaaS版からDb2 Warehouse on Cloudで実施した場合の時間)
import datetime
print("{0} Start.".format(datetime.datetime.now()))
if srs_id is None:
    srs_id = 2011
print("srs_id is " +str(srs_id))
%sql AUTOCOMMIT OFF
sql_flag = '-e'
for i, row in domomo_df.iterrows():
    station_id='D' + str(row['station_id'])
    region_id=row['region_id']
    capacity=row['capacity']
    name=row['name']
    company="DOCOMO"
    lon=row['lon']
    lat=row['lat']

    
    %sql {sql_flag} INSERT INTO SHAREBIKE_STATION (station_id, region_id, capacity, name, company, GEO_POINT) VALUES (:station_id, :region_id, :capacity, :name, :company, ST_Point(:lon, :lat, {srs_id}))
    sql_flag = '-q'
    if sqlcode != 0:
        print("Line {0} Error : sqlcode:{1}, sqlstate:{2}, Message: {3}".format(i, sqlcode, sqlstate, sqlerror))
        break;

    if i%100 == 0:
        %sql COMMIT WORK
        print("{0} {1}/{2}: completed.".format(datetime.datetime.now(), i, len(domomo_df)))

%sql COMMIT WORK
%sql AUTOCOMMIT ON
print("{0} End.".format(datetime.datetime.now()))


#### 件数を確認します。

In [ ]:
%sql select count(*) from SHAREBIKE_STATION

### 5.4 OpenStreet（ハローサイクリング） バイクシェア関連情報  station_informationの取得
[公共交通オープンデータセンター](https://www.odpt.org/)から<br>
- [OpenStreet（ハローサイクリング） バイクシェア関連情報(GBFS形式)](https://ckan.odpt.org/dataset/c_bikeshare_gbfs-openstreet)の[バイクシェア関連情報(GBFS形式 / station_information)](https://ckan.odpt.org/dataset/c_bikeshare_gbfs-openstreet/resource/5455c77d-154d-4b22-b0ca-9ea78209ed76)
をダウンロードし、pandasのDataFrameに入れます。

ドコモのデータの場合と方法は同じです。項目がちょっと違っている程度です。

In [ ]:
#OpenStreet（ハローサイクリング） バイクシェア関連情報  station_information　緯度・経度情報を取得
req_url = 'https://api-public.odpt.org/api/v4/gbfs/hellocycling/station_information.json'

# URLにアクセス
request = requests.get(req_url)

# 必要な部分のみ抜き出し
data_json = json.loads(request.text)['data']['stations']

# pandasのDataFrameに入れる
hello_df = pd.DataFrame(data=data_json)
hello_df

### 5.5 OpenStreet（ハローサイクリング） バイクシェア関連情報 station_informationのSHAREBIKE_STATIONへのインポート


ドコモのデータの場合と方法は同じです。項目がちょっと違っている程度です。<br>
件数が多いので時間が少々かかります。

In [ ]:
# SHAREBIKE_STATIONへのインポート(といいつつINSERT)
# 3-4分かかります(同じリージョンのWatssonStudio SaaS版からDb2 Warehouse on Cloudで実施した場合の時間)
import datetime
print("{0} Start.".format(datetime.datetime.now()))
if srs_id is None:
    srs_id = 2011
%sql AUTOCOMMIT OFF
sql_flag = '-e'
for i, row in hello_df.iterrows():
    station_id='H' + str(row['station_id'])
    capacity=row['vehicle_type_capacity']['num_bikes_limit']
    name=row['name']
    address=row['address']
    url=row['rental_uris']
    company="HELLO"
    lon=row['lon']
    lat=row['lat']

    
    %sql {sql_flag} INSERT INTO SHAREBIKE_STATION (station_id,  capacity, name, address, url, company, GEO_POINT) VALUES (:station_id,  :capacity, :name, :address, :url, :company, ST_Point(:lon, :lat, {srs_id}))
    sql_flag = '-q'
    if sqlcode != 0:
        print("Line {0} Error : sqlcode:{1}, sqlstate:{2}, Message: {3}".format(i, sqlcode, sqlstate, sqlerror))
        break;

    if i%100 == 0:
        if i%1000==0:
            %sql COMMIT WORK
            print("{0} {1}/{2}: commit completed.".format(datetime.datetime.now(), i, len(hello_df)))
        print("{0} {1}/{2}: completed.".format(datetime.datetime.now(), i, len(hello_df)))

%sql COMMIT WORK
%sql AUTOCOMMIT ON
print("{0} End.".format(datetime.datetime.now()))


#### 件数を確認します。

In [ ]:
%sql select count(*) from SHAREBIKE_STATION

# 6. ダミーの売上データ作成

次の「地理空間分析」で売上と場所の関係を地図にマップしてみたいので、ダミーの売上データ作成を作成しておきます。

とりあえずドコモ(ドコモのポートはほとんど都内)とHELLOは東京に住所のあるシェアサイクルポートに絞って、2021年の各月の売り上げを乱数で適当に作ります(ポート数が多いほど金額が高くなる確率が高いようにしてあります)。

<span style="color: red; font-size: 200%;">
2022/09/28のDataBase Dojoの環境で使用している場合は、<br>このデータはあらかじめ準備されていますので、6はSKIPしてください。<br>

- 「[7. DB接続の切断](#db_connect_close)」に進んでください。
</span>

In [ ]:
%%sql
DROP TABLE "STATION_SALES";
CREATE TABLE "STATION_SALES"  (
"STATION_ID" VARCHAR(10 OCTETS) NOT NULL, 
"DATE" VARCHAR(10 OCTETS)  NOT NULL,  
"AMOUNT" DECIMAL(10) ,
PRIMARY KEY (STATION_ID, DATE))
IN "USERSPACE1"  
ORGANIZE BY COLUMN;

In [ ]:
# STATION_SALESへのIダミー売上INSERT
# 3-4分かかります(WatssonStudio SaaS版からDb2 Warehouse on Cloudで実施した場合の時間)
import random
import datetime

#　ダミー売上金額作成関数　引数はシェアサイクルポートのcapacity
def get_dummy_ammount(capa):
    a=random.randint(1, 48)
    return capa * a * 160 * 30

# ドコモと東京に住所のあるシェアサイクルポートのSTATION_ID, capacity　を取得
df_st = %sql SELECT  STATION_ID, capacity FROM SHAREBIKE_STATION WHERE COMPANY='DOCOMO' OR ADDRESS LIKE '東京%'

#　ダミー売上金額 INSERT
print("{0} Start.".format(datetime.datetime.now()))
if srs_id is None:
    srs_id = 2011
%sql AUTOCOMMIT OFF
sql_flag = '-e'
for i, row in df_st.iterrows():
    station_id=row['STATION_ID']
    capa = row['CAPACITY']
    for mon in range(12):
        date = datetime.date(2021, mon+1, 1)
        amount = get_dummy_ammount(capa)
        %sql {sql_flag} INSERT INTO STATION_SALES (station_id, date, amount) VALUES (:station_id, :date, :amount)
        sql_flag = '-q'
        if sqlcode != 0:
            print("Line {0} Error : sqlcode:{1}, sqlstate:{2}, Message: {3}".format(i, sqlcode, sqlstate, sqlerror))
            break;
            
    if i%100 == 0:
        if i%1000==0:
            %sql COMMIT WORK
            print("{0} {1}/{2}: commit completed.".format(datetime.datetime.now(), i, len(df_st)))
        print("{0} {1}/{2}: completed.".format(datetime.datetime.now(), i, len(df_st)))
        
%sql COMMIT WORK
%sql AUTOCOMMIT ON
print("{0} End.".format(datetime.datetime.now()))

### 中身を確認します

In [ ]:
%sql select * from STATION_SALES

<a id="db_connect_close"> </a>
# 7. DB接続の切断

In [ ]:
# DB接続の切断
%sql connect close

これで**「データセットアップ」**は完了です！

次の「地理空間分析」に進みましょう。